# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).



## Library Imports

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks & API Token


In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call


In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 100800792,
 'calculationPrice': 'close',
 'change': 6.1,
 'changePercent': 0.03579,
 'close': 0,
 'closeSource': 'liifoafc',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 178.38,
 'iexCloseTime': 1706314963159,
 'iexLastUpdated': 1677477087701,
 'iexMarketPercent': 0.001599772909442213,
 'iexOpen': 177.79,
 'iexOpenTime': 1715368951987,
 'iexRealtimePrice': 175.09,
 'iexRealtimeSize': 31,
 'iexVolume': 2382,
 'lastTradeTime': 1660224592575,
 'latestPrice': 175.27,
 'latestSource': 'Close',
 'latestTime': 'December 7, 2021',
 'latestUpdate': 1689791674931,
 'latestVolume': None,
 'low': 0,
 'lowSource': None,
 'lowTime': None,
 'marketCap': 286913440113

## Parsing Our API Call
This API call has the metric  the price-to-earnings ratio.



In [4]:
pe_ratio = data['peRatio']
pe_ratio

15.42

## Executing A Batch API Call & Building Our DataFrame



In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,156.10,48.49,N/A
1,AAL,18.66,-3.56,N/A
2,AAP,243.86,24.45,N/A
3,AAPL,174.89,15.81,N/A
4,ABBV,123.04,30.0,N/A
...,...,...,...,...
500,YUM,133.20,25.7,N/A
501,ZBH,133.17,34.46,N/A
502,ZBRA,614.62,38.76,N/A
503,ZION,67.48,6.28,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

## Calculating the Number of Shares to Buy

To do this, we will use the `portfolio_input` function 



In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:1000000


In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,SBAC,317.680,-2261.88,61
1,CRM,202.060,-1165.07,97
2,BMY,65.210,-639.08,300
3,TEL,99.630,-328.9,196
4,GILD,70.140,-318.24,279
5,HPE,9.720,-299.36,2017
6,DRI,88.750,-247.4,220
7,KHC,36.964,-237.94,530
8,DIS,136.510,-221.5,143
9,XRAY,45.210,-203.96,433


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies.  Filtering for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data.

In [10]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [11]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

## Dealing With Missing Data in Our DataFrame



In [12]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,184.500,N/A,60.78,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,304.270,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,292.160,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.96,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,24.270,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.030,N/A,14.36,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,58.150,N/A,33.38,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,35.480,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2988.470,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
325,MXIM,107.310,N/A,34.87,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data 
There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)



In [13]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [14]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

Calculating value score percentiles for every stock in the universe.Specifically,Calculating percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP


In [15]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.825743
1      0.051485
2      0.485149
3       0.29604
4      0.611881
         ...   
500    0.542574
501    0.673267
502    0.750495
503    0.091089
504    0.863366
Name: PE Percentile, Length: 505, dtype: object
0      0.805941
1      0.055446
2      0.588119
3      0.938614
4       0.90495
         ...   
500    0.051485
501    0.306931
502    0.843564
503    0.161386
504    0.940594
Name: PB Percentile, Length: 505, dtype: object
0      0.825743
1      0.045545
2      0.193069
3      0.590099
4           0.6
         ...   
500    0.758416
501    0.529703
502    0.764356
503    0.479208
504    0.926733
Name: PS Percentile, Length: 505, dtype: object
0      0.851485
1      0.023762
2      0.437624
3      0.388119
4      0.334653
         ...   
500    0.708911
501    0.641584
502    0.823762
503    0.071287
504     0.89901
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0      0.851485
1      0.061386
2      0.164356
3      0.667327
4      0.532673
         ...   
5

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,155.20,N/A,49.46,0.825743,9.83,0.805941,7.8300,0.825743,31.242081,0.851485,14.859813,0.851485,N/A
1,AAL,18.46,N/A,-3.64,0.051485,-1.62,0.055446,0.4753,0.045545,-7.303672,0.023762,1.415453,0.061386,N/A
2,AAP,238.43,N/A,24.05,0.485149,4.65,0.588119,1.3700,0.193069,13.271472,0.437624,3.013840,0.164356,N/A
3,AAPL,174.75,N/A,15.90,0.29604,22.40,0.938614,4.0000,0.590099,12.198152,0.388119,9.716103,0.667327,N/A
4,ABBV,123.34,N/A,30.00,0.611881,16.05,0.90495,4.0400,0.6,10.940459,0.334653,7.632576,0.532673,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,132.20,N/A,26.40,0.542574,-5.00,0.051485,6.0100,0.758416,21.759913,0.708911,9.914208,0.683168,N/A
501,ZBH,129.31,N/A,33.35,0.673267,2.18,0.306931,3.5000,0.529703,19.217150,0.641584,6.038216,0.40198,N/A
502,ZBRA,608.40,N/A,39.71,0.750495,11.96,0.843564,6.1800,0.764356,29.397174,0.823762,12.784703,0.784158,N/A
503,ZION,65.12,N/A,6.28,0.091089,1.40,0.161386,3.0100,0.479208,4.924650,0.071287,2.610064,0.128713,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated.


In [16]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,155.20,N/A,49.46,0.825743,9.83,0.805941,7.8300,0.825743,31.242081,0.851485,14.859813,0.851485,0.832079
1,AAL,18.46,N/A,-3.64,0.051485,-1.62,0.055446,0.4753,0.045545,-7.303672,0.023762,1.415453,0.061386,0.047525
2,AAP,238.43,N/A,24.05,0.485149,4.65,0.588119,1.3700,0.193069,13.271472,0.437624,3.013840,0.164356,0.373663
3,AAPL,174.75,N/A,15.90,0.29604,22.40,0.938614,4.0000,0.590099,12.198152,0.388119,9.716103,0.667327,0.57604
4,ABBV,123.34,N/A,30.00,0.611881,16.05,0.90495,4.0400,0.6,10.940459,0.334653,7.632576,0.532673,0.596832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,132.20,N/A,26.40,0.542574,-5.00,0.051485,6.0100,0.758416,21.759913,0.708911,9.914208,0.683168,0.548911
501,ZBH,129.31,N/A,33.35,0.673267,2.18,0.306931,3.5000,0.529703,19.217150,0.641584,6.038216,0.40198,0.510693
502,ZBRA,608.40,N/A,39.71,0.750495,11.96,0.843564,6.1800,0.764356,29.397174,0.823762,12.784703,0.784158,0.793267
503,ZION,65.12,N/A,6.28,0.091089,1.40,0.161386,3.0100,0.479208,4.924650,0.071287,2.610064,0.128713,0.186337


## Selecting the 50 Best Value Stocks¶



In [17]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy


In [18]:
portfolio_input()

Enter the value of your portfolio:1000000


In [19]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

/Users/harikrishnathallapally/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,25.00,800,6.24,0.088119,0.4501,0.057426,0.3705,0.027723,3.021000,0.031683,0.372749,0.007921,0.042574
1,AAL,18.46,1083,-3.64,0.051485,-1.6200,0.055446,0.4753,0.045545,-7.303672,0.023762,1.415453,0.061386,0.047525
2,PRU,109.26,183,5.76,0.081188,0.6634,0.063366,0.5877,0.053465,4.475573,0.059406,0.568689,0.011881,0.053861
3,BEN,33.17,602,4.68,0.071287,0.7456,0.075248,1.0187,0.130693,3.164155,0.033663,0.850231,0.021782,0.066535
4,AIG,56.05,356,8.82,0.136634,0.7250,0.071287,0.9625,0.124752,4.221469,0.053465,0.939623,0.027723,0.082772
5,BA,211.64,94,-14.77,0.041584,-8.8000,0.045545,2.0300,0.319802,-33.627522,0.005941,-1319.943765,0.00198,0.08297
6,ALL,111.45,179,10.51,0.174257,1.2800,0.134653,0.6689,0.063366,2.224269,0.029703,0.634822,0.015842,0.083564
7,TSN,88.10,227,5.07,0.075248,0.8712,0.087129,0.3347,0.017822,4.017648,0.047525,3.469691,0.2,0.085545
8,KSS,53.43,374,8.31,0.124752,1.5000,0.185149,0.3843,0.029703,3.867448,0.043564,1.204273,0.049505,0.086535
9,MET,62.74,318,10.55,0.176238,0.7732,0.077228,0.7464,0.083168,5.242667,0.083168,0.724464,0.017822,0.087525


## Formatting Our Excel Output



In [20]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File


* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric



In [21]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [22]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output


In [23]:
writer.save()